In [76]:
from ase.lattice.cubic import SimpleCubicFactory, BodyCenteredCubicFactory

from ase.lattice.tetragonal import CenteredTetragonalFactory

class Mn2AuFactory(CenteredTetragonalFactory):
    "A factory for creating Mn2Au lattices."
    bravais_basis = [[0.0, 0.0,  0.333333], [0.0 , 0.0, 0.666666], [0, 0, 0]]
    element_basis = (0, 0, 1)

Mn2Au = Mn2AuFactory()

alat = 3.1445
factor = 2.5658

cubic_unit_cell = Mn2Au(["Mn", "Au"], latticeconstant=[alat, factor * alat]) 

print(cubic_unit_cell)

cubic_unit_cell.write("Mn2Au_cubic.xyz")

Lattice(symbols='Mn2AuMn2Au', pbc=True, cell=[3.1445, 3.1445, 8.0681581])


In [77]:
# The lattice units from the paper
6.29 / 1.8897259886

3.3285248961728757

In [78]:
# c/a from the paper
16.142 / 6.291

2.5658877761882053

In [89]:
def make_primitive_Mn2Au_cell(alat, factor):
    
    cell = [[-alat / 2.0, alat / 2.0, alat * factor / 2.0],
            [alat / 2.0, -alat / 2.0, alat * factor / 2.0],
            [alat / 2.0, alat / 2.0, -alat * factor / 2.0]
           ]
    
    Au = Atoms("Au", positions=[[0.0, 0.0, 0.0]], cell=cell, pbc=True)
    Mn2 = Atoms("Mn2", positions=[[0.333, 0.333, 0.0], [0.6666, 0.666, 0.0]], cell=cell, pbc=True)

    return Au + Mn2


In [90]:
primitive_unit_cell = make_primitive_Mn2Au_cell(alat, factor)
print(primitive_unit_cell)

Atoms(symbols='AuMn2', pbc=True, cell=[[-1.57225, 1.57225, 4.03407905], [1.57225, -1.57225, 4.03407905], [1.57225, 1.57225, -4.03407905]])


In [91]:
primitive_unit_cell.write("Mn2Au_primitive.xyz")

In [98]:
from ase.calculators.vasp import Vasp

n_par = 10

VaspCalc = Vasp(setups={'W': '_sv'}, # Check pseudopotentials for Mn and Au here
                npar=n_par, kpts=[3,3,3],
                istart=0, # start from scratch
                icharg=2, # default for istart=0
                isif=2, 
                nsim=2,
                prec='Accurate',
                encut=500,
                ediff=1.e-6,
                ediffg=-0.04,
                nelm=100,
                nelmin=5,
                algo="Normal",
                isym=0,
                ismear=1,
                sigma=0.1,
                ispin=2, # spin polarised calculation
                magmom=[0.0, 2.0, -2.0], # a first guess for the Mn2Au
                xc='PBE')

In [106]:
VaspCalc.write_incar(primitive_unit_cell)